# Agent 와 MCP

## Agent
  * use LLMs to generate ‘thoughts’ bases on ‘observations’ to perform ‘actions’

### [Agent 역할](https://huggingface.co/learn/agents-course/unit1/what-are-agents)
  * 자연어 이해
    * 인간의 지시 해석/응답
  * 추론 및 계획
    * 정보 분석, 결정, 문제 해결 전략 수립
  * 툴과 상호작용
    * 여러 툴을 이용해 정보 수집, 행동, 그 행동의 결과 관찰

### Agent 구성

#### [LLMs \(Model\)](https://huggingface.co/learn/agents-course/unit1/what-are-llms) 

* Agent 뇌 - 추론, 계획, 의사 결정
* 인간의 언어 이해/생성
* Objective
  * EOS 전까지 이전의 토큰을 기반으로 다음 토큰 예측
* Prompting the LLM
  * 다음 토큰을 예측할때 주어지는 가이드

In [ ]:
# !pip install langchain langchain-mcp-adapters langchain-openai langgraph

In [1]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

#### [Tools](https://huggingface.co/learn/agents-course/unit1/tools)

* Agent 몸 - 행동 실행
* LLM은 외부 데이터나 연산을 할 수 없음
* LLM 에게 주어지는 **함수**
  * 도구의 기능에 대한 텍스트 설명
  * 호출 가능한 함수
  * 입력 인자와 타입
  * 출력과 타입


In [ ]:
def calculator(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b

"""
Tool Name: calculator, Description: Multiply two integers., 
Arguments: a: int, b: int, Outputs: int
"""

In [2]:
from langchain_core.tools import tool

import requests

In [3]:
NWS_API_BASE = "https://api.weather.gov"

def make_nws_request(url: str) -> dict[str] | None:
    """Make a request to the NWS API with proper error handling."""
    headers = {"User-Agent": "weather-app/1.0", "Accept": "application/geo+json"}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except Exception:
        return None

@tool
def get_forecast(latitude: float, longitude: float) -> str:
    """Get weather forecast for a location.

    Args:
        latitude: Latitude of the location
        longitude: Longitude of the location
    """
    points_url = f"{NWS_API_BASE}/points/{latitude},{longitude}"
    points_data = make_nws_request(points_url)

    if not points_data:
        return "Unable to fetch forecast data for this location."

    forecast_url = points_data["properties"]["forecast"]
    forecast_data = make_nws_request(forecast_url)

    if not forecast_data:
        return "Unable to fetch detailed forecast."

    periods = forecast_data["properties"]["periods"]
    forecasts = []
    for period in periods[:1]:
        forecast = (
            f"{period['name']}:"
            f"Temperature: {period['temperature']}°{period['temperatureUnit']}"
            f"Wind: {period['windSpeed']} {period['windDirection']}"
            f"Forecast: {period['detailedForecast']}"
        )
        forecasts.append(forecast)
    return "\n---\n".join(forecasts)

In [4]:
tools = [get_forecast]

  * LLM & Tool 동작
    * 사용자 질문 -> LLM 이 호출할 Tool / 입력 인자 생성 -> Tool 호출 / 출련 반환 -> LLM 응답 생성

  * Tool 디자인
    * 툴들을 정해놓고 원하는 툴을 선택
    * 툴을 직접 만들어내는 툴을 호출

  * MCP, Model Context Protocol
    * Tool 반복해서 구현하는 대신 재사용할 수 있도록 도구 제공 방식을 표준화한 프로토콜


### [AI Agent Workflow](https://huggingface.co/learn/agents-course/unit1/agent-steps-and-structure)


<img width="500" alt="Image" src="https://github.com/user-attachments/assets/73984383-cfc1-4e10-8db8-869dbb1d4af5" />

  * Thought
    * LLM이 다음 스텝을 결정
  * Action
    * Tool 실행
  * Observation
    * Tool 응답 반영


In [5]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(model, tools)

In [6]:
for s in agent.stream({"messages": "New York 날씨는?"}, stream_mode="values"):
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

================================ Human Message =================================

New York 날씨는?
================================== Ai Message ==================================
Tool Calls:
  get_forecast (call_JtXwfDBPbHonB9ZpCMDor59o)
 Call ID: call_JtXwfDBPbHonB9ZpCMDor59o
  Args:
    latitude: 40.7128
    longitude: -74.006
================================= Tool Message =================================
Name: get_forecast

Today:Temperature: 80°FWind: 10 to 23 mph SWForecast: A slight chance of rain showers before 8am, then a slight chance of showers and thunderstorms. Partly sunny, with a high near 80. Southwest wind 10 to 23 mph, with gusts as high as 39 mph. Chance of precipitation is 20%.
================================== Ai Message ==================================

뉴욕의 오늘 날씨는 다음과 같습니다:

- **기온:** 약 80°F (약 27°C)
- **바람:** 남서쪽에서 시속 10에서 23마일까지, 최대 돌풍은 시속 39마일까지
- **날씨 예보:** 오전 8시 이전에 약간의 비 소나기가 올 가능성이 있으며, 이후 약간의 비와 뇌우가 있을 수 있습니다. 구름이 조금 낄 것으로 예상되며 강수 확률은 20%입니다.



#### [Thought](https://huggingface.co/learn/agents-course/unit1/thoughts)
  * LLM이 다음 스텝을 결정
  * Agent 내부 추론/계획 프로세스 
    * 현재까지 정보로 다음 행동 결정
      * “사용자가 이전에 Python을 선호한다고 했으니, 예제를 Python으로 제공해야겠다.”
    * 복잡한 문제를 작은 단계로 나누기
      * “이 작업을 완료하려면 1) 데이터 수집, 2) 분석, 3) 보고서 작성의 단계가 필요하다.”
    * 과거 관찰로 전략 수정
      * “이전 접근 방식이 효과적이지 않았으니 다른 전략을 시도해야겠다.”
  * ReAct Aproach
    * Reasoning & Acting
    * 행동하기 전에 단계별로 동작하도록 유도하는 프롬프트 기법
      * 마지막에 ‘Let’s think hink step by step’ 추가
      * 한번에 결론 내는 대신 계획을 만들고 하나씩 해결
      * 모델 Fine-tuning 에 "think before answering" 추가하고 <think>와 </think> 구각 포함해서 학습 - Deepseek R1 / OpenAI's o1

#### [Action](https://huggingface.co/learn/agents-course/unit1/actions)
  * Tool 실행
  * Agent 가 수행하는 구체적인 작업
  * Stop and Parse Aproach
    * 구조화된 형식으로 생성
    * 불필요한 토큰 생성 중단
    * 출력 중 호출할 도구와 파라미터만 파싱


#### [Observation](https://huggingface.co/learn/agents-course/unit1/observations)
  * Tool 응답 반영
  * Agent Action으로 얻은 피드백
    * 행동 성공 여부나 결과 데이터 피드백 수집
    * 결과 통합
    * 다음 생각 전략 조정

    
#### Thought-Action-Observation Cycle
  * End or Another thought?
  * 목표 달성 전까지 Cycle 반복
  * 툴 활용, LLM이 정적인 지식을 넘어서 실시간 데이터 활용
  * 동적 조절, 정확한 정답이 나올때 까지 진행

#### 만약에
* LLM 이 멍청하면? 정확한지 판단을 할 수 없음
* Tool 이 부족하면? 행동할 수 없음

In [7]:
for s in agent.stream({"messages": "뉴욕 날씨랑 서울 날씨 비교해줘. 서울 날씨 수집이 안되면 LA 랑 비교해"}, stream_mode="values"):
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

================================ Human Message =================================

뉴욕 날씨랑 서울 날씨 비교해줘. 서울 날씨 수집이 안되면 LA 랑 비교해
================================== Ai Message ==================================
Tool Calls:
  get_forecast (call_vp4Jik4vqfuhN0OaWg9KAwTe)
 Call ID: call_vp4Jik4vqfuhN0OaWg9KAwTe
  Args:
    latitude: 40.7128
    longitude: -74.006
  get_forecast (call_BrTNeEfwSbpyRVBZ4FMad63q)
 Call ID: call_BrTNeEfwSbpyRVBZ4FMad63q
  Args:
    latitude: 37.5665
    longitude: 126.978
================================= Tool Message =================================
Name: get_forecast

Unable to fetch forecast data for this location.
================================== Ai Message ==================================
Tool Calls:
  get_forecast (call_k6SKTliHZRE8OCoeLnAYFQgu)
 Call ID: call_k6SKTliHZRE8OCoeLnAYFQgu
  Args:
    latitude: 34.0522
    longitude: -118.2437
================================= Tool Message =================================
Name: get_forecast

Overnight:Tempera

### [Agentic System](https://www.anthropic.com/engineering/building-effective-agents)
  * Workflow
    * LLM 과 Tool 이 이미 정의된 방식으로 동작
    * ex) RAG
      * 쿼리 기반 DB 검색 후 요약 (싱글 스텝)
  * Agent
    * LLM 이 직접 목표 달성을 위해 어떤 플로우로 Tool 을 사용할 것인지 지시
    * ex) Agnetic RAG
      * 쿼리 기반 DB 검색 후 정보가 부족하면 다른 쿼리로 재질의 후 요약
      * Query Reformulation, Result Validation, Multi-Step Retrieval, Source Integration




# [MCP](https://modelcontextprotocol.io/introduction)

* Tool 반복해서 구현하는 대신 재사용할 수 있도록 도구 제공 방식을 표준화한 프로토콜 ?


* Model Context Protocol
  * Model: LLM
  * Context: 에이전트가 작업을 수행할 때 참고하는 정보
  * Protocol: 의사소통 규칙


* LLM 과 데이터, Tool 간 통합을 표준화하는 개방형 프로토콜 (USB-C)
  * Resources, Prompts, Tools

* MCP 역할
  * 컨텍스트 공유
  * 모듈 간 일관성 유지
  * 의사결정 흐름 정렬


* MCP Architecture
  * MCP Host
    * Claude Desktop, Cursor
    * MCP 를 통해 데이터에 접근하는 프로그램
  * MCP Server
    * MCP 를 통해 기능을 노출하는 프로그램


## MCP Server Concepts



In [24]:
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("weather")

if __name__ == "__main___":
    mcp.run()

### Resources

* LLM 이 읽을 수 있는 데이터 정보 노출

In [20]:
@mcp.resource("file:///logs/app.log") # log files, images, database, ...
def read_log_file() -> str:
    """Read Log file"""
    return "Log File ~~~"

### Promts

* LLM Intercation 패턴 정의 

In [22]:
from mcp.server.fastmcp.prompts import base

@mcp.prompt()
def review_code(code: str) -> str:
    return f"Please review this code:\n\n{code}"


@mcp.prompt()
def debug_error(error: str) -> list[base.Message]:
    return [
        base.UserMessage("I'm seeing this error:"),
        base.UserMessage(error),
        base.AssistantMessage("I'll help debug that. What have you tried so far?"),
    ]

### Tools

* 외부 작업을 수행할 수 있게 하는 기능 제공

In [21]:
import requests

@mcp.tool()
async def fetch_weather(city: str) -> str:
    """Fetch current weather for a city"""
    response = requests.get(f"https://api.weather.com/{city}")
    return response.text

### [Best Practice](https://modelcontextprotocol.io/tutorials/building-mcp-with-llms)

* 복잡한 기능은 나눠서 구현
* 각 컴포넌트별 테스트 철저히
* 보안 고려: 입력 검증 및 접근 제한
* 코드 문서화
* MCP 스펙에 맞게 구현

### MCP 설정
* ```python 
    # server.py

    from mcp.server.fastmcp import FastMCP

    mcp = FastMCP("weather")

    mcp.tool(get_forecast)
    mcp.run(transport="sse") # "stdio"
    ```

### MCP Server 실행
* ```bash
    python server.py
    ```

## MCP Client

### with Cursor

```json
{
  "mcpServers": {
    "server-name-stdio": {
      "command": "python",
      "args": ["server.py"]
    },
    "server-name-sse": {
      "url": "http://localhost:3000/sse"
    }
  }
}
```

### with Langchain

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "server-name-sse": {
        "url": "http://localhost:8000/sse",
        "transport": "sse",
    }
})
_ = await client.__aenter__()

Error in sse_reader: peer closed connection without sending complete message body (incomplete chunked read)


In [ ]:
tools = client.get_tools()
tools

[StructuredTool(name='get_forecast', description='Get weather forecast for a location.\n\n    Args:\n        latitude: Latitude of the location\n        longitude: Longitude of the location\n    ', args_schema={'properties': {'latitude': {'title': 'Latitude', 'type': 'number'}, 'longitude': {'title': 'Longitude', 'type': 'number'}}, 'required': ['latitude', 'longitude'], 'title': 'get_forecastArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x10c8fbce0>)]

In [ ]:
agent = create_react_agent(model, client.get_tools())

In [ ]:
async for s in agent.astream({"messages": "뉴욕 날씨 알려줘"}, stream_mode="values"):
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

================================ Human Message =================================

뉴욕 날씨 알려줘
================================== Ai Message ==================================
Tool Calls:
  get_forecast (call_iNBEdgb81u4ukMQKGsmN1cTn)
 Call ID: call_iNBEdgb81u4ukMQKGsmN1cTn
  Args:
    latitude: 40.7128
    longitude: -74.006
================================= Tool Message =================================
Name: get_forecast

Today:Temperature: 80°FWind: 10 to 23 mph SWForecast: A slight chance of rain showers before 8am, then a slight chance of showers and thunderstorms. Partly sunny, with a high near 80. Southwest wind 10 to 23 mph, with gusts as high as 39 mph. Chance of precipitation is 20%.
================================== Ai Message ==================================

오늘 뉴욕의 날씨는 다음과 같습니다:

- **온도:** 80°F
- **바람:** 서남서 방향으로 10에서 23 mph 속도의 바람이 붑니다.
- **날씨 예보:** 오늘 오전 8시 전에는 비가 약간 올 가능성이 있으며, 그 후에는 소나기와 천둥번개가 약간 있을 수 있습니다. 대체로 맑음이며, 최고 기온은 약 80°F입니다. 바람은 서남서 방향으로 최대 39 mph의 강풍이 불 수 있

## [MCP Official Servers](https://github.com/modelcontextprotocol/servers)

* [Github](https://github.com/modelcontextprotocol/servers/tree/main/src/github)
* [ElasticSearch](https://github.com/elastic/mcp-server-elasticsearch)
* [Notion](https://github.com/makenotion/notion-mcp-server#readme)